## Practice Lab: Linear Regression

Welcome to your first lab in today's workshop! In this lab, you will build a training model to predict whether an image is a bird or not. 

## Outline
- [ 1: Collecting our Data ](#t1)
- [ 2: Problem Statement ](#t2)
- [ 3: Dataset ](#t3)
- [ 4: Training our Model ](#t4)
- [ 5: Testing our Model (and building your own!) ](#t5)
    

_**NOTE:** This lab was sourced from the University of Queensland's free online course **Fast.ai**._
_You can learn more from https://course.fast.ai/_

<a name="t1"></a>
## 1: Collecting our Data

First, let's run the cell below to download and import all the packages you will need for this lab. If you want to uninstall these packages at the end of the lab, just replace `install` with `uninstall`.
- [fastai](https://pypi.org/project/fastai/) is a package created by Fast.ai that condenses most of the Pytorch libraries you need.
- [duckduckgo_search](https://pypi.org/project/duckduckgo-search/) helps us search and download images to our local working directory

In [ ]:
# Download packages for this lab
import sys
!{sys.executable} -m pip install fastai
!{sys.executable} -m pip install fastcore
!{sys.executable} -m pip install fastdownload
!{sys.executable} -m pip install duckduckgo_search

In [ ]:
# Import more modules
from duckduckgo_search import ddg_images
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *

<a name="t2"></a>
## 2: Problem Statement

Imagine that you are a birdkeeper who has accumulated millions of photographs of your zoo over time. 
- Unfortunately, you have reached your storage limit and must now decide which photos to keep. 
- Fortunately, many of the pictures were taken haphazardly and only feature empty shots of trees, so you can delete those without hesitation. 
- Unfortunately, going through each photo individually to delete them is not a desirable option.

How can we get machine learning to categorize and delete these pictures for us?


<a name="t3"></a>
## 3: Dataset

We will start this task by downloading a dataset from an online search engine. For this lab, we will directly download from DuckDuckGo.com

The first step will be to define a function that can download images and put them into our working file directory for us.
- The `search_images()` function will be used to collect the url of the images that match the 'image' description we provide.

    - `term` is the keyword of our search 
    - `max_results` is the maximum amount of images we want the function to fetch


In [ ]:
# Define a function to search images for us
def search_images(term, max_images=50):
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

##### View your variables
To familiarize yourself with your dataset, it is a good practice to view your variables before beginning any task. 

- Since we have a function that extracts the necessary information from DuckDuckGo, a good place to start is to download a sample image by executing the code block below. 

- After running the cell below, you'll notice that the same image displayed below has been saved inside the same folder where you saved this file as bird_test.jpg!

_**Note**: Feel free to change the term 'birds' to something else if you prefer!_

In [ ]:
# Fetch the url of an image search for 'birds'
urls = search_images('birds', max_images=1)

# Set a name for the image in your file
dest = 'bird_test.jpg'

# Download the image of the url
download_url(urls[0], dest, show_progress=False)

# Open the image to see our picture
Image.open(dest).to_thumb(256,256)

In order to create our model, it's necessary to obtain images of photos without birds. Unfortunately, this is hard to search for, so we wil use trees as the pictures that are not birds. Similar to the previous step, we can use the following code block to retrieve a single forest photo. It's the same thing, just shortened and condensed into two lines. Go ahead and execute it.

In [ ]:
download_url(search_images('forest', max_images=1)[0], 'forest.jpg', show_progress=False)
Image.open('forest.jpg').to_thumb(256,256)

##### Downloading our Dataset
Assuming that the results you got in the steps before are reasonable, we can move forward with the data collection process. As a next step, we can create a local folder in our directory and obtain 50 images of both birds and not birds(forests). These can be easily deleted later after we are done with the lab.

By gathering additional data, we can further improve the accuracy and reliability of our model, ensuring that it can make more precise predictions and be better suited to real-world applications. So, let's proceed with collecting these images and taking our model to the next level!

In [ ]:
searches = 'birds','forest'
# Creates a new folder in our current working directory
path = Path('dataset')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o}', max_images=50))
    resize_images(path/o, max_size=400, dest=path/o)

<a name="t4"></a>
## 4. Training our Model

Great job! We have successfully downloaded all 100 of the images and are now ready to begin training our model. However, before we proceed, it's essential to note that some of the downloaded images may not have been retrieved correctly, and their presence can potentially hinder our model training process.

Therefore, we need to review the folder containing our downloaded images and remove any files that are corrupted or have not downloaded correctly. This critical step ensures that our model training process proceeds without any obstacles, enabling us to construct a robust and reliable model that can classify images with precision and accuracy.

##### Cleaning up our data
To ensure that our model remains functional and reliable in the long run, the code below identifies and removes any invalid image files from our dataset. This step is crucial because such files can cause errors or complications during the model training process.

_**NOTE**: In addition, it's worth noting that not all images in our dataset may be relevant or appropriate for our classification purposes. For instance, some images may depict objects resembling birds, such as bird-patterned lamps, which can potentially confuse our model. Therefore, it's advisable to review our dataset and remove any irrelevant images to enhance the accuracy and efficiency of our model._

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)

len(failed)

To develop and train our model, we require DataLoaders - an object that encompasses a training set (comprising images utilized for constructing a model) and a validation set (comprising images employed to verify the model's accuracy, which are not utilized during the training process).

Thankfully, creating DataLoaders in fastai is a straightforward task that involves using a DataBlock. Through this mechanism, we can generate a DataBlock that incorporates all the necessary components to construct and train our model. Furthermore, we can even view a few sample images from our DataBlock to ensure that it contains the desired data and is properly formatted.

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),             # The inputs to our model are Images, and outputs are Categories

    get_items=get_image_files,                      # The find all inputs (images), we run get_image_files to return a                                                 # list of all image files in the path

    splitter=RandomSplitter(valid_pct=0.2, seed=42), # Randomly split our data, so we can keep 20% of images to check
                                                     # our model later after the model is built.

    get_y=parent_label,                             # The labels are the name of our parent folder (bird or forest)

    item_tfms=[Resize(192, method='squish')]        # Before we train anything, we force all images to be 192 x 192
                                                    # pixels by squishing it (not cropping!)
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)                             # Show six random images

We have completed all the necessary preparation steps and are now ready to commence training our model. The fastest computer vision model that is widely used is resnet18. Remarkably, even on a CPU, it's possible to train this model in just a few minutes. However, with a GPU, the training time can be further reduced to under 10 seconds.

In fastai, there is an incredibly useful method called fine_tune() that automates the process of fine-tuning a pre-trained model. This method employs best practices to ensure that the model is fine-tuned correctly, significantly enhancing its accuracy and efficiency. Therefore, we will utilize the fine_tune() method to train our model and ensure that it is optimized to deliver precise predictions.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

<a name="t5"></a>
## 5. Testing our Model (and building your own!)

Now we can test out our model by downloading our own images and putting it into the model! We can test our model in many ways, including:

- Download images of birds from the internet, and saving them to our device.
- Taking our own pictures and uploading them to our device.

_**Note**: Make sure to save the image in the same file that this Jupyter Notebook is in, and rename it into_ `bird_test.jpg` _(or change the code below to match the image file)_

In [ ]:
is_bird, _, probs = learn.predict(PILImage.create('bird_test.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability that the model thinks it is a bird: {probs[0]:.4f}")

##### Cleanup

Once you're finished, you may not want to keep these python packages used today on your local machine. As suck, you can optionally uninstall the packages by running the code below.

In [ ]:
# Optional: Uninstall installed packages for this lab
!{sys.executable} -m pip uninstall fastai
!{sys.executable} -m pip uninstall fastcore
!{sys.executable} -m pip uninstall fastdownload
!{sys.executable} -m pip uninstall duckduckgo_search